In [1]:
import pandas as pd
import numpy as np
import random
import glob
import itertools

In [2]:
#read orginal dataset downloaded from GPCRdb 
Chemokine = pd.read_csv("ChemokineReceptor.csv",sep=';')
Chemokine = Chemokine[['ChEMBL ID', 'Receptor', 'Smiles']]
Chemokine = Chemokine.rename(columns = {'ChEMBL ID':'ChEMBL_ID'})

In [3]:
df = Chemokine[['ChEMBL_ID','Receptor']]

In [5]:
receptors= Chemokine.Receptor.unique()
ligands= Chemokine.ChEMBL_ID.unique()

In [6]:
ligands.shape

(8296,)

In [7]:
receptors.shape

(39,)

In [8]:
#find all GPCR pairs which don't have overlap ligands
df_receptors_comb = pd.DataFrame(columns = ['Receptor_X', 'Receptor_Y'])
for i in range(0,len(receptors)):    #len(receptors)
    non_overlap_rep = []
    df_X = df[df['Receptor'] == receptors[i]]
    #df_tmp = df[df['Receptor'] != receptors[0]]
    df_tmp = df[~df.Receptor.isin(list(receptors[range(0,i+1)]))]  
    other_repcetor = df_tmp.Receptor.unique()

    for j in range(38 - i):
        #df_tmp = df[~df.Receptor.isin(list(receptors[range(0,i+1)]))]    
        df_Y = df_tmp[df_tmp['Receptor'] == other_repcetor[j]]
        if (sum(df_X.ChEMBL_ID.isin(df_Y.ChEMBL_ID) ) == 0):
            non_overlap_rep.append(other_repcetor[j])
    
    rep_X = np.repeat(receptors[i], len(non_overlap_rep))
    d = {'Receptor_X':rep_X,'Receptor_Y':non_overlap_rep}
    df_comb = pd.DataFrame(d)
    df_receptors_comb = df_receptors_comb.append(df_comb)

In [9]:
Non_overlap_ReceptorX = df_receptors_comb.Receptor_X.unique()

In [10]:
####for each receptor X, generate negative data using receptorX + Ligands from receptorY 
#####receptorY is the receptor has no same ligands with receptorX
all_negative_dataset = pd.DataFrame(columns = ['negative_rep', 'negative_ligand'])
for i in range(0,len(Non_overlap_ReceptorX)):
    df_comb_sub = df_receptors_comb[df_receptors_comb['Receptor_X'] == Non_overlap_ReceptorX[i]]
    df_original_sub = df[df.Receptor.isin(df_comb_sub.Receptor_Y)]
    ligand_select = df_original_sub.ChEMBL_ID.unique()
    negative_rep = np.repeat(Non_overlap_ReceptorX[i], len(ligand_select))
    d0 = {'negative_rep':negative_rep,'negative_ligand':ligand_select}
    negative_df = pd.DataFrame(d0)
    all_negative_dataset = all_negative_dataset.append(negative_df)

In [11]:
all_negative_dataset.shape

(24489, 2)

In [12]:
random.seed(8)
negative = all_negative_dataset.sample(n = 9166)

In [13]:
negative = negative[['negative_ligand','negative_rep']]
negative = negative.rename(columns = {'negative_ligand':'ChEMBL_ID', 'negative_rep' : 'Receptor'})

In [14]:
positive = df

In [15]:
pos_neg = positive.append(negative)

In [16]:
pos_neg.shape

(18332, 2)

In [17]:
receptor= df.Receptor.unique()
ligand = df.ChEMBL_ID.unique()
df_long = pd.DataFrame(list(itertools.product(ligand,receptor)),columns=['ChEMBL_ID','Receptor'])

In [18]:
df_merge_pos = pd.merge(df_long, positive, on = ['ChEMBL_ID','Receptor'], how = 'left',indicator = 'pos_exist')
df_merge_pos['pos_exist'] = np.where(df_merge_pos.pos_exist == 'both', 1, 2)

In [19]:
df_merge_pos['pos_exist'].value_counts()

2    314378
1      9166
Name: pos_exist, dtype: int64

In [20]:
df_merge_pos_neg = pd.merge(df_merge_pos, negative, on = ['ChEMBL_ID','Receptor'], how = 'left',indicator = 'neg_exist')
df_merge_pos_neg['neg_exist'] = np.where(df_merge_pos_neg.neg_exist == 'both', 0, 2)

In [21]:
df_merge_pos_neg['neg_exist'].value_counts()

2    314378
0      9166
Name: neg_exist, dtype: int64

In [22]:
df_merge_pos_neg.loc[(df_merge_pos_neg['pos_exist'] == 1) & (df_merge_pos_neg['neg_exist'] == 2), 'indicator'] = 1
df_merge_pos_neg.loc[(df_merge_pos_neg['pos_exist'] == 2) & (df_merge_pos_neg['neg_exist'] == 0), 'indicator'] = 0
df_merge_pos_neg.loc[(df_merge_pos_neg['pos_exist'] == 2) & (df_merge_pos_neg['neg_exist'] == 2), 'indicator'] = ''

In [23]:
df_merge_pos_neg1 = df_merge_pos_neg.drop(columns = ['pos_exist','neg_exist'])

In [24]:
df_wide= df_merge_pos_neg.pivot(index='ChEMBL_ID', columns='Receptor', values='indicator')

In [25]:
smile=Chemokine[['ChEMBL_ID','Smiles']].drop_duplicates()

In [26]:
df_wide = pd.merge(df_wide,smile, on=['ChEMBL_ID'], how='inner')

In [27]:
df_wide.shape

(8296, 41)

In [28]:
#df_wide.to_csv("/Users/Xuenan/Documents/UIUC/2020Fall/cs412/Project/Input_pos_neg.csv",index=False)

In [29]:
df1 = df_wide.drop(['ChEMBL_ID', 'Smiles'], axis = 1)
df2 = df_wide[['ChEMBL_ID','Smiles']]

In [30]:
aa = df1.apply(pd.value_counts) #count postive and negtive number of each receptor
bb = aa.T #transpose the matrix, making GPCR names in row
bb.rename(columns = {0.0: 'count0'},inplace = True)
bb.rename(columns = {1.0: 'count1'},inplace = True)

In [31]:
bb.loc[(bb['count0'] > 10) & (bb['count1'] > 10), 'indicator'] = 'True' #select the receptors inclduing >10 pos and >10 neg
cc = bb.indicator[bb.indicator == 'True'].index.tolist() # get the list of select receptors
dd = df1[cc] # filter the dataframe based on the list,25 receptors satisfy the condition

In [32]:
df_c = pd.concat([dd, df2], axis=1)
df_c.shape

(8296, 27)

In [33]:
df_c.to_csv('Input_Pos10Neg10.csv',index = False) ##generate the final input dataset, including 25 GPCRs and 8296 ligands

In [51]:
random.seed(30)
#split input dataset into training dataset and testing dataset based on the proportion 4:1
train_dataset = df_c.sample(n=6636)
train_dataset.to_csv('train_dataset.csv',index = False)

In [52]:
test_dataset = df_c.drop(train_dataset.index)
test_dataset.to_csv('test_dataset.csv',index = False)